<a href="https://colab.research.google.com/github/hwata3535/KataGo/blob/master/colab_katago.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

이 문서는 Google Colaboratory에 카타고를 세팅하고 리지나 사바키(또는 다른 GTP 엔진을 지원하는 도구)와 연결하는 방법을 설명합니다.

# 1단계 : Colab 서버에서 KataGo와 SSH 실행하기

실행하기 전에 NGROK_TOKEN 값을 입력해 주어야 합니다.

PC마다 값이 다르기 때문에 리지를 실행하시는 컴퓨터가 바뀐 경우는 재입력 하셔야 합니다.

아래의 링크를 눌러서 이동하여 맨 위에 Copy 버튼을 누르시면 토큰값이 복사됩니다.

[https://dashboard.ngrok.com/auth/your-authtoken](https://dashboard.ngrok.com/auth/your-authtoken).

그 다음 아래 스크립트 영역 맨 윗줄의 입력하는 곳에 붙여넣기 하시면 됩니다.

원하시는 경우에는 USER_NAME과 USER_PASSWORD 값도 바꾸실 수 있습니다.(이 경우 나중에 PC의 리지 설정에서도 똑같이 바꿔주어야 합니다.)

그 다음에는 아래에서 실행버튼(동그란 아이콘)을 눌러서 실행하시면 됩니다. 

전체 스크립트가 실행되는 데 1~2분 정도 걸립니다.

참고로, 자동으로 서버에 할당된 GPU가 CUDA를 지원하지 않는 경우 처음 리지를 실행했을 때 OpenCL 튜닝 절차가 진행될 것입니다.

CUDA 버전이 선택된 경우 대략 NVIDIA RTX 2060 정도의 성능이 나오는 듯 합니다.

In [3]:
NGROK_TOKEN="1oJTj1YTdq5ELgP4Cll85j0SjY7_5e9JRZ9MNinLHsZRiv3Pd"
USER_NAME="katago"
USER_PASSWORD="20b"

# optional args
# supports: OPENCL, CUDA or AUTO
KATAGO_BACKEND="AUTO"
# supports: 40b, k15, 20b, 40b-large or AUTO
WEIGHT_FILE="20b" 

import subprocess
if KATAGO_BACKEND == "AUTO":
  gpu_name=str(subprocess.check_output("nvidia-smi -q | grep \"Product Name\" | cut -d\":\" -f2 | tr -cd '[:alnum:]._-'", shell=True), encoding='utf-8')
  if gpu_name == "TeslaV100-SXM2-16GB":
    KATAGO_BACKEND="OPENCL"
  else:
    KATAGO_BACKEND="OPENCL"
!echo "Using Katago Backend: " $KATAGO_BACKEND
!echo "Using Katago Weight: " $WEIGHT_FILE
!echo "GPU : " $gpu_name

weight_urls = {
    '40b': 'https://github.com/lightvector/KataGo/releases/download/v1.4.5/g170-b40c256x2-s5095420928-d1229425124.bin.gz',
    '30b': 'https://github.com/lightvector/KataGo/releases/download/v1.4.5/g170-b30c320x2-s4824661760-d1229536699.bin.gz',
    '20b': 'https://github.com/lightvector/KataGo/releases/download/v1.4.5/g170e-b20c256x2-s5303129600-d1228401921.bin.gz',
    '40b-large': 'https://github.com/kinfkong/katago-colab/releases/download/v1.4.5/40b384.bin.gz'
}

# Install useful stuff
! apt-get update 1>/dev/null
! apt install --yes ssh screen nano htop ranger git libzip4 1>/dev/null
! pip install oss2 1>/dev/null
%cd /content

!rm -rf katago-colab
!git clone https://github.com/kinfkong/katago-colab.git 1>/dev/null

#download the binarires
!wget --quiet https://github.com/kinfkong/katago-colab/releases/download/v1.4.5/katago-$KATAGO_BACKEND -O katago
!chmod +x /content/katago
!wget --quiet https://github.com/kinfkong/katago-colab/releases/download/v1.4.5/ngrok -O ngrok
!chmod +x /content/ngrok
!mkdir -p /root/.katago/
!cp -r /content/katago-colab/opencltuning /root/.katago/

#download the weights
weight_url = weight_urls[WEIGHT_FILE]
!wget --quiet $weight_url -O $WEIGHT_FILE".bin.gz" 
!rm -rf weight.bin.gz
!ln -s $WEIGHT_FILE".bin.gz" weight.bin.gz

# SSH setting
! echo "root:$USER_PASSWORD" | chpasswd
! echo "PasswordAuthentication yes" > /etc/ssh/sshd_config
! echo "PermitUserEnvironment yes" >> /etc/ssh/sshd_config
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config

! mkdir -p /root/.ssh

# generate the keys
#!ssh-keygen -q -t rsa -N '' -f /root/.ssh/id_rsa <<<y 2>&1 >/dev/null
#!cat /root/.ssh/id_rsa.pub >> /root/.ssh/authorized_keys

! service ssh restart > /dev/null

# Run ngrok
get_ipython().system_raw('./ngrok authtoken $NGROK_TOKEN && ./ngrok tcp 22 &')
! sleep 5

import oss2
import requests
import json
from re import sub
r = requests.get('http://localhost:4040/api/tunnels')
raw_ssh = r.json()['tunnels'][0]['public_url']
ssh_args = (sub("tcp://", "", raw_ssh)).split(':')
ssh_option = {
    'host': ssh_args[0],
    'port': int(ssh_args[1]),
    'user': 'root'
}

ssh_option_json = json.dumps(ssh_option)

endpoint = 'http://oss-cn-beijing.aliyuncs.com'
auth = oss2.Auth('LTAI4G3tthX2R3Z8KY6Bdbs3', 'Q22UstMdKX8zZY9BqkGWWd2XbEBvPH')
bucket = oss2.Bucket(auth, endpoint, 'kata-config')

key = USER_NAME + '.ssh.json'
bucket.put_object(key, ssh_option_json)

with open('/content/katago-colab/config/gtp_colab.cfg', mode='w') as f:
    f.write('logDir = gtp_logs\n')
    f.write('logAllGTPCommunication = true\n')
    f.write('logSearchInfo = true\n')
    f.write('logToStderr = false\n')
    f.write('ogsChatToStderr = true\n')
    f.write('analysisWideRootNoise = 0.04\n')
    f.write('rules = korean\n')
    f.write('defaultBoardSize = 19\n')
    f.write('allowResignation = true\n')
    f.write('resignThreshold = -0.90\n')
    f.write('resignConsecTurns = 3\n')
    f.write('avoidMYTDaggerHack = true\n')
    f.write('ponderingEnabled = true\n')
    f.write('lagBuffer = 0.1\n')
    f.write('numSearchThreads = 32\n')
    f.write('nnMaxBatchSize = 16\n')
    f.write('searchFactorAfterOnePass = 0.50\n')
    f.write('searchFactorAfterTwoPass = 0.25\n')
    f.write('searchFactorWhenWinning = 0.40\n')
    f.write('searchFactorWhenWinningThreshold = 0.95\n')

!echo -e "\n[KataGo Config]" 
!cat /content/katago-colab/config/gtp_colab.cfg
!echo -e "\n"

!echo "done!"


Using Katago Backend:  OPENCL
Using Katago Weight:  20b
GPU :  TeslaV100-SXM2-16GB


/content
Cloning into 'katago-colab'...
remote: Enumerating objects: 102, done.
remote: Total 102 (delta 0), reused 0 (delta 0), pack-reused 102
Receiving objects: 100% (102/102), 32.42 KiB | 851.00 KiB/s, done.
Resolving deltas: 100% (46/46), done.
ngrok: Text file busy

[KataGo Config]
logDir = gtp_logs
logAllGTPCommunication = true
logSearchInfo = true
logToStderr = false
ogsChatToStderr = true
analysisWideRootNoise = 0.04
rules = korean
defaultBoardSize = 19
allowResignation = true
resignThreshold = -0.90
resignConsecTurns = 3
avoidMYTDaggerHack = true
ponderingEnabled = true
lagBuffer = 0.1
numSearchThreads = 32
nnMaxBatchSize = 16
searchFactorAfterOnePass = 0.50
searchFactorAfterTwoPass = 0.25
searchFactorWhenWinning = 0.40
searchFactorWhenWinningThreshold = 0.95


done!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 2단계 : Colab KataGo를 사바키 또는 리지에 연결하기<br>(세븐틴이 배포한 패키지 안에는 이미 세팅이 완료되어 있음)

세븐틴 배포 패키지 다운로드 :
http://naver.me/58hVIZEz

## 1) 먼저 아래에서 colab-katago 프로그램을 다운로드합니다.
다운로드 랑크:  
**For Windows Users (64bit windows)**  
https://github.com/kinfkong/katago-colab/releases/download/v1.4.5/colab-katago.windows.zip  
**For Linux Users**  
https://github.com/kinfkong/katago-colab/releases/download/v1.4.5/colab-katago.linux.zip  
**For Mac OSX Users**  
https://github.com/kinfkong/katago-colab/releases/download/v1.4.5/colab-katago.mac.zip  


**다운로드하신 후에 압축파일 안에 있는 colab-katago 또는 colab-katago.exe 파일을 리지 폴더에 복사해 넣습니다.**

## 2) 다음, 사바키 또는 리지에 엔진을 등록합니다.

To configure the engine in `Sabaki` or `Lizzie`, you just need to fill the absolute path of your `colab-katago` program (which you've downloaded just now), and the username, password(i.e, the `USER_NAME`, `USER_PASSWORD` you configured in Step 1).


**Sabaki Example**:  

![Sabaki Example Image](
https://kata-config.oss-cn-beijing.aliyuncs.com/images/Snip20200725_12.png
)  

**Lizzie Example**:  

![Lizzie Example Image](https://kata-config.oss-cn-beijing.aliyuncs.com/images/Snip20200725_14.png
)

## 추가 설정(선택)
사바키에서 visits 또는 착수시간을 지정할 수 있습니다.(아래 참조)

```
<AbsolutePathOfColabKataGoProglem> <USER_NAME> <USER_PASSWORD> 30s
```
30초로 착수시간 지정
```
<AbsolutePathOfColabKataGoProglem> <USER_NAME> <USER_PASSWORD> 1600v
```
1600 visits 지정